In [8]:
import pandas as pd
import numpy as np
import json
import os
import plotly.offline as py
import plotly.graph_objs as go
import IPython.display
from IPython.display import Image

# Initialize the python notebook mode for plotly
py.init_notebook_mode(connected=True)

In [9]:
# Load the json metadata file
file_path = 'data/nmr_metadata.json'
with open(file_path, 'r') as f:
    metadata = json.load(f)

In [10]:
print(metadata['studies'][0]['publications'][0]['title'])
metadata['studies'][0]['publications'][0]['doi']

$^{27}	ext{Al}$ NMR and Raman spectroscopic studies of alkaline 	       aluminate solutions with extremely high caustic content               - Does the octahedral species $	ext{Al(OH)}_6^{-3}$ exist in               solution?


'10.1016/j.talanta.2006.02.008'

In [11]:
# Ask this json object to load the data files therein.
# TODO: How will this scale? Does this allow for multiple data files?
for df in metadata['studies'][0]['assays'][0]['dataFiles'][:]:
    print('type:\t\t{0}'.format(df['type']))
    print('name:\t\t{0}'.format(df['name']))
    
# In this demo case, simply assign these manually
demo_csv = metadata['studies'][0]['assays'][0]['dataFiles'][1]['name']

type:		Plot-image
name:		data/sipos2006_fig2.png
type:		Plot-csv-extract
name:		data/sipos2006-fig2.csv


In [12]:
# read the .csv in the local data folder
sipos_2006_nmr = pd.read_csv(demo_csv)  # That was easy and worked well...
# This should have returned a dataframe object

In [13]:
# Show the head of the table
sipos_2006_nmr.head()

,Al(III) M,NaOH M,27 Al ppm
0,0.005,1.0,80.45
1,0.005,5.0,80.40
2,0.005,7.0,79.59
3,0.005,11.0,78.13
4,0.005,13.0,76.93


In [14]:
# Get the unique aluminate concentrations
# unique_conc = sipos_2006_nmr['Al(III) M'].unique()
# Create new data frames for each
# traces = []
# for uc in unique_conc:
    

In [15]:
traces = []
for conc, value in sipos_2006_nmr.groupby('Al(III) M'):
    traces.append(value)

In [16]:
# now add the desired metadata to the table
# doing it in this way separates our data & metadata in how we input the data
sipos_2006_nmr['text'] = ['[Al(III)] = {}'.format(x) for x in sipos_2006_nmr['Al(III) M']]
# sipos_2006_nmr['title'] = metadata['studies'][0]['publications'][0]['title']
# sipos_2006_nmr['doi'] = metadata['studies'][0]['publications'][0]['doi']

In [17]:
# Use some empty annotations to place a hyperlink...
plotAnnotes = []

plotAnnotes.append(dict(x=sipos_2006_nmr['NaOH M'],
                        y=sipos_2006_nmr['27 Al ppm'],
                        text="""<a href="https://google.com/">{}</a>""".format("Text"),
                        showarrow=True,
                        xanchor='center',
                        yanchor='center',
                        ))

In [18]:
# Create the new plot/figure with the axis names and figure title.

layout = go.Layout(title='27 Al NMR',
                   yaxis=dict(title='27 Al ppm'),
                   xaxis=dict(title='[NaOH]')
)

data = []

for t in traces:
    n_str = '{} Al(III) M'.format(t.iloc[0][0])
    new_trace = go.Scatter(
        name=n_str,
        x = t['NaOH M'],
        y = t['27 Al ppm'],
        mode='markers',
        text=sipos_2006_nmr['text'],
    )
    data.append(new_trace)

fig = go.Figure(data=data, layout=layout)
plot1 = py.iplot(fig, filename='scatter-for-dashboard')